In [1]:
!pip install transformers timm einops

In [ ]:
!pip install pandas

In [ ]:
import numpy as np
import pandas as pd
import os

pd.set_option('display.max_colwidth', 256)
import re
import numpy as np 
# from https://www.kaggle.com/code/nilaychauhan/fine-tune-gemma-models-in-keras-using-lora

# # Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [ ]:
!pip install -U "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

In [ ]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3
!export KERAS_BACKEND="jax"

In [ ]:
!export XLA_PYTHON_CLIENT_MEM_FRACTION=1.00

In [ ]:
!pip install --upgrade keras

In [ ]:
import keras
print(keras.__version__)

In [ ]:
import keras
import keras_nlp
#keras.mixed_precision.set_global_policy('mixed_bfloat16') ## this gives out of memory error
keras.config.set_floatx("bfloat16")


In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en") 

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import torch
# Check for CUDA availability and set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

model_id = "vikhyatk/moondream2"
revision = "2024-03-06"
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, revision=revision).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)

cuda


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
prompt= ""
def classific(path):
    global prompt
    image = Image.open(path)
    enc_image = model.encode_image(image)
    category= model.answer_question(enc_image, """ accurately categorize image name among the following classes:
    battery, biological , glass ,cardboard, clothes, metal , paper, plastic , shoes , edible_waste.

    #.And show the category in single word and donot give any explanation . """, tokenizer)
    print(category)

    if category == "Glass":
        prompt = """Please provide detailed information on the proper disposal methods for glass waste, including recycling processes and environmentally friendly disposal techniques. 
        Additionally, suggest practical strategies and initiatives that individuals and communities can implement to reduce their carbon footprint. This may include tips for energy conservation, sustainable transportation options, waste reduction practices, and other eco-friendly lifestyle changes.
"""
    elif category == "Paper":
        prompt = """Please provide detailed information on the proper disposal methods for Paper waste, including recycling processes and environmentally friendly disposal techniques.

Additionally, suggest practical strategies and initiatives that individuals and communities can implement to reduce their carbon footprint. This may include tips for energy conservation, sustainable transportation options, waste reduction practices, and other eco-friendly lifestyle changes.
"""   
    elif category == "Cardboard":
        prompt = """Please provide detailed information on the proper disposal methods for Cardboard waste, including recycling processes and environmentally friendly disposal techniques.

Additionally, suggest practical strategies and initiatives that individuals and communities can implement to reduce their carbon footprint. This may include tips for energy conservation, sustainable transportation options, waste reduction practices, and other eco-friendly lifestyle changes.
""" 
    elif category == "Plastic":
        prompt = """Please provide detailed information on the proper disposal methods for Plastic waste, including recycling processes and environmentally friendly disposal techniques.

Additionally, suggest practical strategies and initiatives that individuals and communities can implement to reduce their carbon footprint. This may include tips for energy conservation, sustainable transportation options, waste reduction practices, and other eco-friendly lifestyle changes.
"""
    elif category == "Metal":
        prompt = """Please provide detailed information on the proper disposal methods for Metal waste, including recycling processes and environmentally friendly disposal techniques.

Additionally, suggest practical strategies and initiatives that individuals and communities can implement to reduce their carbon footprint. This may include tips for energy conservation, sustainable transportation options, waste reduction practices, and other eco-friendly lifestyle changes.
"""  
    else:
        prompt = "Please provide information about recycling."

# Print the prompt
 
    
# Put you inmagd
classific("garbage classification/Garbage classification/glass/glass10.jpg")



Glass


In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import os
import torch

# Check for CUDA availability and set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model and tokenizer
model_id = "vikhyatk/moondream2"
revision = "2024-03-06"
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, revision=revision).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)

def classific(path):
    image = Image.open(path).convert('RGB')  # Ensure image is in RGB
    enc_image = model.encode_image(image).to(device)  # Move encoded image to GPU
    category = model.answer_question(enc_image, """
        accurately categorize image name among the following classes:
        glass, cardboard, metal, paper, plastic.

        #.And show the category in single word and do not give any explanation.
    """, tokenizer)
    return category.strip().lower()  # Return category as lowercase for uniformity

# Define the directories and materials
root_dir = 'garbage classification/Garbage classification'
materials = ["glass", "cardboard", "metal", "paper", "plastic"]
accuracy_dict = {material: {'correct': 0, 'total': 0} for material in materials}

# Iterate over each material directory
for material in materials:
    print(material)
    directory = os.path.join(root_dir, material)
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            image_path = os.path.join(directory, filename)
            predicted_category = classific(image_path)
            # Increment counts
            accuracy_dict[material]['total'] += 1
            if material == predicted_category:
                accuracy_dict[material]['correct'] += 1

# Calculate and print accuracy for each material
for material, counts in accuracy_dict.items():
    if counts['total'] > 0:
        accuracy = counts['correct'] / counts['total']
        print(f"Accuracy for {material}: {accuracy:.2f}")



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


glass
cardboard
metal
paper
plastic
Accuracy for glass: 0.99
Accuracy for cardboard: 0.84
Accuracy for metal: 0.40
Accuracy for paper: 0.80
Accuracy for plastic: 0.68
